In [1]:
import sys
sys.path.append("src")

In [2]:
from imutils.video import FileVideoStream
from tqdm import tqdm
import pickle
from facial_analysis import *
from torch.utils.data import DataLoader, Dataset

In [3]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [4]:
path = "Engagement/FaceEngageDataset/u1/u1_00001.mp4"

In [5]:
video = FileVideoStream(path).start()

frame_idx = []
faces = []
i = 0
while True:
    frame = video.read()
    if frame is None:
        break
    face = detect_face(frame)
    if face is not None:
        faces.append(clean_face(face))
        frame_idx.append(i)
    i += 1

results = {"Emotions": [], "Valence": [], "Arousal": []}
faces_dataset = MyDataset(torch.stack(faces))
faces_dataloader = DataLoader(faces_dataset, batch_size=16, shuffle=False)
with torch.no_grad():
    for batch in faces_dataloader:
        out = net(batch)
        expr = torch.softmax(out["expression"], dim=-1)
        results["Emotions"].append(expr)
        results["Valence"].append(out["valence"])
        results["Arousal"].append(out["arousal"])

for k, v in results.items():
    results[k] = np.array(torch.cat(v).cpu())
results["Frame Index"] = frame_idx

In [6]:
len(frame_idx), results["Emotions"].shape, results["Valence"].shape

(264, (264, 8), (264,))